### 1- import libraries

In [1]:
from bs4 import BeautifulSoup
import requests 
import time
import datetime
import smtplib

### 2- requests handling

In [4]:
URL ='https://www.amazon.co.uk/Amazon-Brand-Sensitive-Unscented-wipes/dp/B07V2N4SJY/ref=zg_bs_g_baby_d_sccl_1/257-9862628-6554964?psc=1'
headers = 'User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'